In [ ]:
#I don't think I'll need a new indicator for customer variable or for new run. 
    #think order_id and an indicator in days_since prior order will suffice

#customer level data
#order number, order_dow, order_hour_of_day,days_since_prior_order
#need to substitue a value for days_since_prior_order where it is an NA

#product level data
#product_id, aisle_id, department_id
    #not doing reordered or add_to_cart_order as RNN can pick up on those the way I'm doing batches
    #unsure how to work in aisle_id and department_id to the embeddings
    #thinking I should initialize them as random uniform around those clusters
        #maybe one hot the aisle and department id, let the rest be random uniform
        #do that for like first 10th of embedding dim be department_id, 2nd 10th be aisle_id

#to do
#df_orders: drop order_id, user_id, eval_set but still want to be able to link them
#guess I need some sort of get batch function that iters over the user_id and order number to get this info.
#ie given the user_id,

#products
#do the advanced embedding (ie with department and aisle id) if you can get the rest set up
#need to set up the inputs and the targets
#want an end of order character
#so flip, transpose, and add a end of order character
#work this into the get_batches thing

#uncertain of:
#memory constraints: can i hold all the files as loaded as I get bachtes through them? hopefully yes

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_orders = pd.read_csv("orders.csv/orders.csv")
df_prior = pd.read_csv("order_products__prior.csv/order_products__prior.csv")
df_train = pd.read_csv("order_products__train.csv/order_products__train.csv")

#df_products = 
#df_aisles = pd.read_csv("aisles.csv/aisles.csv")
#df_departments = pd.read_csv("departments.csv/departments.csv")

In [3]:
df_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
df_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [40]:
df_prior['product_id'][df_prior['order_id'] == 2]

0    33120
1    28985
2     9327
3    45918
4    30035
5    17794
6    40141
7     1819
8    43668
Name: product_id, dtype: int64

In [5]:
df_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [12]:
#df_train[df_train['order_id'] == 1]
#df_prior[df_prior['order_id'] == 2]
#df_orders[df_orders['order_id'] == 2]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3355525,2,202279,prior,3,5,9,8.0


In [64]:
#test = [1,[5,1,6,9,0]]
test = []
test.append([1,5,1,6,9,0])
test.append([1,5,1,6,9,0])
#test = [1,5,1,6,9,0]
print(test)
a = np.asarray(test,dtype=np.int32)
np.savetxt("foo.csv", a, delimiter=",",fmt='%i')

[[1, 5, 1, 6, 9, 0], [1, 5, 1, 6, 9, 0]]


In [ ]:
#max for priors days before is 30, so for first order saying it is -30



In [ ]:
#creating three data frames:
#inputs part 1:
    #non products, basically df_orders
    #products_input: basically transpose df_prior and df_train. want order id as the index with each product_id its own column
    #products_target: like products_input but lined up so that each is a sequential response to the prior order
    
    

In [ ]:
#non_products
df_orders_train = pd.read_csv("orders.csv/orders.csv")
df_orders_train = df_orders_train.fillna(-30)

z_break = 0
user_ids = df_orders_train['user_id'].unique()
print(len(df_orders_train))

for i in user_ids:
#     z_break += 1
#     if z_break > 10:
#         break
    #print(i,len(df_orders_train))
    if len(df_orders_train[(df_orders_train['user_id'] == i) & (df_orders_train['eval_set'] == 'test')]) == 1:
        df_orders_train = df_orders_train[df_orders_train['user_id'] != i]
        #print(i,len(df_orders_train))
        
print(len(df_orders_train))
df_orders_train.to_csv('input_orders_train.csv',index=False)

3421083


In [ ]:
df_orders_test = pd.read_csv("orders.csv/orders.csv")
df_orders_test = df_orders_test.fillna(-30)

z_break = 0
user_ids = df_orders_test['user_id'].unique()
print(len(df_orders_test))

for i in user_ids:
#     z_break += 1
#     if z_break > 10:
#         break
    #print(i,len(df_orders_test))
    if len(df_orders_test[(df_orders_test['user_id'] == i) & (df_orders_test['eval_set'] == 'train')]) == 1:
        df_orders_test = df_orders_test[df_orders_test['user_id'] != i]
        #print(i,len(df_orders_test))
        
print(len(df_orders_test))
df_orders_test.to_csv('input_orders_test.csv',index=False)

In [31]:
#df_orders.to_csv('input_orders.csv',index=False)

206209


In [130]:
# user_id = df_orders['user_id'].unique()
# print(len(user_id))
# user_batch_orders = df_orders[(df_orders['user_id'] == 1) & (df_orders['eval_set'] == "train")]
# print(user_batch_orders)

    order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
10   1187899        1    train            11          4                  8   

    days_since_prior_order  
10                    14.0  


In [162]:
#products input and targets
#for training set want both
#for test set, just want inputs
import pickle

user_ids = df_orders['user_id'].unique()

current_user_type = None #train or test
current_train_batch_count = 0
current_test_batch_count = 0
train_batch_number = 0
test_batch_number = 0
input_products = []
target_products = []
test_products = []
z_break = 0

for i in user_ids:
    
#     z_break += 1
#     if z_break > 10:
#         break
        
    current_input = []
    current_target = []
    current_batch = []
    
    if len(df_orders[(df_orders['user_id'] == i) & (df_orders['eval_set'] == 'train')]) == 1:
        current_user_type = "train"
        current_train_batch_count += 1
    else:
        current_user_type = "test"
        current_test_batch_count += 1
        
    if current_train_batch_count > 1000:
        train_batch_number += 1
        current_train_batch_count = 0
        pickle.dump( np.array(input_products), open( "train_batch_" + str(train_batch_number) +  "_input_products.p", "wb" ) )
        pickle.dump( np.array(target_products), open( "train_batch_" + str(train_batch_number) + "_target_products.p", "wb" ) )
        input_products = []
        target_products = []
        
    if current_test_batch_count > 1000:
        test_batch_number += 1
        current_test_batch_count = 0
        pickle.dump( np.array(test_products), open( "test_batch_" + str(test_batch_number) +  "_input_products.p", "wb" ) )
        test_products = []
    
    if current_user_type == "train":
        user_batch_orders = df_orders['order_id'][df_orders['user_id'] == i].tolist()
    #    user_batch_orders = df_orders[df_orders['user_id'] == i]
        for j in range(0,len(user_batch_orders)):
            if j == 0:
                products = df_prior['product_id'][df_prior['order_id'] == user_batch_orders[j]]
                current_input = products.tolist()
                current_input.append(0)
                current_input.insert(0,user_batch_orders[j])
            else:
                if j == len(user_batch_orders) - 1:
                    products = df_train['product_id'][df_train['order_id'] == user_batch_orders[j]]
                else:
                    products = df_prior['product_id'][df_prior['order_id'] == user_batch_orders[j]]
                current_target = products.tolist()
                current_target.append(0)
                current_target.insert(0,user_batch_orders[j])
                
                input_products.append(current_input)
                target_products.append(current_target)
                current_input = current_target
    else:
        user_batch_orders = df_orders['order_id'][(df_orders['user_id'] == i) & (df_orders['eval_set'] == "prior")].tolist()
        for j in range(0,len(user_batch_orders)):
            products = df_prior['product_id'][df_prior['order_id'] == user_batch_orders[j]] 
            current_input = products.tolist()
            current_input.append(0)
            current_input.insert(0,user_batch_orders[j])
            test_products.append(current_input)
    
#print(input_products)
#print(target_products)
#print(test_products)
        
pickle.dump( np.array(input_products), open( "train_batch_" + str(train_batch_number) +  "_input_products.p", "wb" ) )
pickle.dump( np.array(target_products), open( "train_batch_" + str(train_batch_number) + "_target_products.p", "wb" ) )
pickle.dump( np.array(test_products), open( "test_batch_" + str(test_batch_number) +  "_input_products.p", "wb" ) )

In [163]:
#print(len(input_products),len(target_products))
#print(df_train[df_train['order_id']==1492625])
#print(test[j])
#user_batch_orders

In [168]:
# pickle.dump( np.array(input_products), open( "train_batch_" + str(train_batch_number) +  "_input_products.p", "wb" ) )
# pickle.dump( np.array(target_products), open( "train_batch_" + str(train_batch_number) + "_target_products.p", "wb" ) )
# pickle.dump( np.array(test_products), open( "test_batch_" + str(test_batch_number) +  "_input_products.p", "wb" ) )

#test1 = pickle.load(open("train_batch_" + str(train_batch_number) +  "_input_products.p","rb")) #pickle.load( open( "image_count_dict.p", "rb" ) )
#print(test1)
#test2 = pickle.load(open("train_batch_" + str(train_batch_number) + "_target_products.p","rb")) #pickle.load( open( "image_count_dict.p", "rb" ) )
#print(test2)
#test3 = pickle.load(open("test_batch_" + str(test_batch_number) +  "_input_products.p","rb")) #pickle.load( open( "image_count_dict.p", "rb" ) )
#print(test3)


[[2398795, 196, 10258, 12427, 13176, 26088, 13032, 0]
 [473747, 196, 12427, 10258, 25133, 30450, 0]
 [2254736, 196, 12427, 10258, 25133, 26405, 0]
 [431534, 196, 12427, 10258, 25133, 10326, 17122, 41787, 13176, 0]
 [3367565, 196, 12427, 10258, 25133, 0]
 [550135, 196, 10258, 12427, 25133, 13032, 0]
 [3108588, 12427, 196, 10258, 25133, 46149, 49235, 0]
 [2295261, 49235, 46149, 25133, 196, 10258, 12427, 0]
 [2550362, 196, 46149, 39657, 38928, 25133, 10258, 35951, 13032, 12427, 0]
 [1187899, 196, 25133, 38928, 26405, 39657, 10258, 13032, 26088, 27845, 49235, 46149, 0]
 [1501582, 16797, 47526, 8479, 47766, 19051, 8138, 0]
 [1901567, 47766, 32792, 20574, 7781, 28874, 0]
 [738281, 49451, 32792, 32139, 34688, 36735, 37646, 22829, 24852, 47209, 33276, 45613, 9681, 21150, 0]
 [1673511, 47144, 5322, 17224, 38656, 21376, 48210, 5907, 14553, 47553, 22124, 19156, 1559, 47766, 0]
 [1199898, 32792, 12000, 9681, 32139, 32052, 27344, 24852, 45066, 27966, 47209, 36735, 46676, 33754, 17872, 18523, 24954,

In [161]:
# #products input and targets
# #saving in batches of 1000 customer ids
# import pickle

# user_id = df_orders['user_id']
# order_ids = df_orders['order_id']
# eval_set = df_orders['eval_set']

# input_products = []
# target_products = []
# len_orders = len(order_ids)
# z_break = 0

# current_input = []
# current_target = []
# current_user_id = None
# current_user_type = None #train or test
# current_batch_count = 0
# batch_number = 0


# for i in range(0,len_orders-1):
#     z_break += 1
#     if z_break > 100:
#         break
    
#     if user_id[i] != current_user_id:
#         current_user_id = user_id[i]
#         #see if customer is in the train or test set
#         if len(df_orders['user_id'][df_orders['eval_set'] == 'train']) == 1:
#             current_user_type = "train"
#         else:
#             current_user_type = "test"
#         current_batch_count += 1
        
#     if current_batch_count > 1000:
#         batch_number += 1
#         current_batch_count = 0
#         pickle.dump( np.array(input_products), open( "batch_" + str(batch_number) +  "_input_products.p", "wb" ) )
#         pickle.dump( np.array(target_products), open( "batch_" + str(batch_number) + "_target_products.p", "wb" ) )
    
    
#     if i == 0:
#         if eval_set[i] == 'prior':
#             products = df_prior['product_id'][df_prior['order_id'] == order_ids[i]]
#         elif eval_set[i] == 'train':
#             products = df_train['product_id'][df_train['order_id'] == order_ids[i]]
#         current_input = products.tolist()
#         current_input.append(0)
#         current_input.insert(0,order_ids[i])
        
#     else:
#         current_input = current_target
#         current_target = []
        
#     #current_target.append(order_ids[i+1])
#     if eval_set[i] == 'prior':
#         products = df_prior['product_id'][df_prior['order_id'] == order_ids[i+1]]
#     elif eval_set[i] == 'train':
#         products = df_train['product_id'][df_train['order_id'] == order_ids[i+1]]
    
#     current_target = products.tolist()
#     current_target.append(0)
#     current_target.insert(0,order_ids[i+1])
    
#     input_products.append(current_input)
#     target_products.append(current_target)
    
# #saves the last batch
# pickle.dump( np.array(input_products), open( "batch_" + str(batch_number) +  "_input_products.p", "wb" ) )
# pickle.dump( np.array(target_products), open( "batch_" + str(batch_number) + "_target_products.p", "wb" ) )

# #print(input_products)
# #print(target_products)
# #print(np.array(input_products))
# #print(np.array(target_products))

In [ ]:
# import pickle

# pickle.dump( np.array(input_products), open( "input_products.p", "wb" ) )
# pickle.dump( np.array(target_products), open( "target_products.p", "wb" ) )


In [ ]:
#DONE NEED TO CREATE EQUIVALENT SHIT FOR TEST SET
#DONE MIGHT WANT TO ADD CUSTOMER NUMBER OF ORDERS AS A VARIABLE SOMEWHERE BASED ON HOW I"M DOING BATCHES
    #not needed, linked through order ids
#DONE I FUCKED UP, FOR THE NEXT CUSTOMER I CAN'T USE THE PRIOR CUSTOMERS LAST ORDER

In [115]:
# for i in range(1,10):
#     print(test1[i][0],test2[i-1][0])
#     #print(test2[i+1][0])
#     if test1[i][0] == test2[i-1][0]:
#         print("test")
    

In [ ]:
#ip_array = np.asarray(input_products)#np.asarray(test,dtype=np.int32)
#print(ip_array)
#print(ip_array.shape)
#np.savetxt("input_products.csv", ip_array, delimiter=",")

# tp_array = np.asarray(target_products,dtype=np.int32)
# np.savetxt("target_products.csv", tp_array, delimiter=",",fmt='%i')